In [1]:
import csv
import numpy as np
import pandas as pd 

In [2]:
working_dir = '../data/processed_pd_data'

In [3]:
pd_df = pd.read_csv(working_dir + '/merged_pd_data.csv')

In [4]:
pd_df.head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority,neighborhood,call_type_desc,crime_type,crime_focus,date_time_bin
0,E22010000006,2022-01-01 00:01:14,7,4600,NaN,ZION,AVE,NaN,NaN,NaN,415,K,321,2.0,Grantville,DISTURBING PEACE,minor,person,Late Night
1,E22010000007,2022-01-01 00:01:23,7,6700,NaN,KELLY,ST,NaN,NaN,NaN,PARTY,K,621,3.0,Linda Vista,LOUD PARTY,minor,property,Late Night
2,E22010000009,2022-01-01 00:02:06,7,4100,NaN,IOWA,ST,NaN,NaN,NaN,415V,K,813,1.0,North Park,DISTURBING PEACE W/VIOLENCE,violence,person,Late Night
3,E22010000012,2022-01-01 00:03:41,7,4600,NaN,35TH,ST,NaN,NaN,NaN,246R,R,811,3.0,Normal Heights,FIRING AT OCCP'D HSE/VEH 1110,violence,person,Late Night
4,E22010000014,2022-01-01 00:04:38,7,0,NaN,INFO LOG,NaN,NaN,NaN,NaN,503CAR,W,-1,2.0,NaN,STOLEN VEHICLE LOG,theft_robbery,property,Late Night


In [5]:
print("Unique Inc. Nos.",len(pd.unique(pd_df['incident_num'])))

Unique Inc. Nos. 103230


In [6]:
len(pd_df)

103250

In [7]:
# Creating the Crime nodes
duplicate = pd_df[pd_df['incident_num'].duplicated()]
crime_node_df = pd_df[~pd_df['incident_num'].duplicated()].copy()
crime_node_df['priority'] = crime_node_df['priority'].astype(int)

**Creating node file for crime/incident_no.**

In [9]:
#Converting priority  to int
crime_node_df["node_type"] = "CRIME_INCIDENT_NO"
crime_node_list = crime_node_df[['incident_num','priority','crime_type','node_type']].values.tolist()

crime_node_header = ["incident_no:ID", "priority:Integer","crime_type:String",":LABEL"]
nodes_crime_type_file = working_dir +'/nodes/nodes_crime_type.csv'
# write nodes file 
with open(nodes_crime_type_file, 'w',  newline='') as f:
    writer = csv.writer(f)
    writer.writerow(crime_node_header)
    writer.writerows(crime_node_list)

**Creating node file for beat/neighborhood**

In [10]:
# Creating the neighborhood nodes
neighborhood_node_df = pd_df[['beat','neighborhood']].dropna().drop_duplicates().copy()

#Creating a serial ID for the dataframe
#neighborhood_node_df['ID'] = np.arange(len(neighborhood_node_df))

# To start from ID = 1 
#neighborhood_node_df['ID'] = neighborhood_node_df['ID'] + 1
#neighborhood_node_df = neighborhood_node_df.iloc[:,[1,0]]
neighborhood_node_df['label'] = 'neighborhood'
neighborhood_node_list = neighborhood_node_df.values.tolist()

neighborhood_node_header = [":ID", "neighborhood",":LABEL"]
neighborhood_file = working_dir +'/nodes/nodes_neighborhood.csv'
# write nodes file 
with open(neighborhood_file, 'w',  newline='') as f:
    writer = csv.writer(f)
    writer.writerow(neighborhood_node_header)
    writer.writerows(neighborhood_node_list)

In [11]:
# Creating the time nodes
time_node_df = pd_df[['date_time_bin']].drop_duplicates().copy()

#Creating a serial ID for the dataframe
time_node_df['ID'] = np.arange(len(time_node_df))

# To start from ID = 1 
time_node_df['ID'] = time_node_df['ID'] + 1
time_node_df = time_node_df.iloc[:,[1,0]]
time_node_df['label'] = 'time'
time_node_list = time_node_df.values.tolist()

time_node_header = [":ID", "time",":LABEL"]
time_node_file = working_dir +'/nodes/node_time.csv'
# write nodes file 
with open(time_node_file, 'w',  newline='') as f:
    writer = csv.writer(f)
    writer.writerow(time_node_header)
    writer.writerows(time_node_list)

**Create Crime happened at neighbourhood relationships**

In [13]:
crime_loc_df = crime_node_df[['incident_num','beat','neighborhood']].dropna().drop_duplicates().copy()
crime_loc_df['type'] = 'HAPPENED_IN'
crime_loc_header = [':START_ID',':END_ID','neighborhood',':TYPE']
crime_loc_list = crime_loc_df.values.tolist()

crime_loc_file = working_dir +'/relationships/crime_loc_rel.csv'

with open(crime_loc_file, 'w',  newline='') as f:
    writer = csv.writer(f)
    writer.writerow(crime_loc_header)
    writer.writerows(crime_loc_list)